<a href="https://colab.research.google.com/github/adnanagbaria/deepLearning/blob/main/classification1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI Classification
This file shows an example of building a classification model

In [ ]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.models import resnet50, ResNet50_Weights
import os

In [ ]:
# Mount to Google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Implement load_dataset() using AI.

AI query: implement a function for loading dataset and return dataset and dataLoader

In [ ]:
# prompt: implement a function for loading dataset and return dataset and dataLoader

def load_dataset(data_dir, batch_size=32):

  # Define the transforms for the images
  transform = transforms.Compose([
      transforms.Resize(256),
      transforms.CenterCrop(224),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
  ])

  # Load the datasets
  train_dataset = ImageFolder(os.path.join(data_dir, 'train'), transform=transform)
  test_dataset = ImageFolder(os.path.join(data_dir, 'val'), transform=transform)

  # Create the data loaders
  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
  test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

  return train_loader, test_loader


In [ ]:
# prompt: Implement a function to show dataset

import matplotlib.pyplot as plt
def show_dataset(dataset, num_images=5):
  # Get a batch of images and labels
  images, labels = next(iter(dataloader))

  # Show the images
  for i in range(num_images):
    plt.imshow(images[i].permute(1, 2, 0))
    plt.title(f"Label: {labels[i]}")
    plt.show()



In [ ]:
# prompt: write a function to initialize the ResNet50 model

def initialize_model():
  # Load the ResNet50 model
  model = resnet50(weights=ResNet50_Weights.DEFAULT)

  # Freeze the model parameters
  for param in model.parameters():
    param.requires_grad = False

  # Modify the final layer to have the correct number of outputs
  num_features = model.fc.in_features
  model.fc = nn.Linear(num_features, len(dataset.classes))

  return model


In [ ]:
# prompt: Write a function to train resnet50

def train_model(model, dataloader, num_epochs=3):
  # Define the loss function and optimizer
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model.to(device)

  # Train the model
  for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(dataloader):
      # Move the images and labels to the device
      images = images.to(device)
      labels = labels.to(device)

      # Forward pass
      outputs = model(images)
      loss = criterion(outputs, labels)

      # Backward pass
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      # Print the progress
      if i % 100 == 0:
        print(f"Epoch: {epoch+1}/{num_epochs}, Batch: {i+1}/{len(dataloader)}, Loss: {loss.item():.4f}")

  # Save the model
  torch.save(model.state_dict(), 'model.pt')



# Main
Here we have the main code
1. Load the dataset
2. Show the dataset (some of them if needed)
3. Initialize the model
4. train the model

In [ ]:
dataset_dir = '/content/drive/MyDrive/Colab Notebooks/deepLearning/dataset/covid'

In [ ]:
# prompt: Load the dataset of train and test

train_loader, test_loader = load_dataset(dataset_dir)


In [ ]:
# prompt: print the size of train dataset

print(f"Train dataset size: {len(train_loader.dataset)}")


Train dataset size: 79


In [ ]:
# prompt: Initialize the model

model = initialize_model()


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 101MB/s]


In [ ]:
# prompt: Train the model

train_model(model, train_loader, num_epochs=3)


Epoch: 1/3, Batch: 1/3, Loss: 0.6805
Epoch: 2/3, Batch: 1/3, Loss: 0.6241
Epoch: 3/3, Batch: 1/3, Loss: 0.5471
